In [10]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [11]:
def do_canny(image):
    gray_scale = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
    
    blur = cv.GaussianBlur(gray_scale, (5,5),0)
    
    canny = cv.Canny(blur, 50, 150)
    return canny

In [12]:
def do_segment(image):
    height = image.shape[0]
    
    polygons = np.array([(0,height), (800,height), (380,290)])
    
    mask = np.zeros_like(image)
    
    cv.fillPoly(mask, [polygons], 255)
    segmented = cv.bitwise_and(image,mask)
    return segmented

In [13]:
def calculate_lines(image, lines):
    left = []
    right = []
    
    for line in lines:
        x1,y1,x2,y2 = line.reshape(4)
        parameters = np.polyfit((x1,x2), (y1,y2), 1)
        slope = parameters[0]
        y_inter = parameters[1]
        
        if slope< 0:
            left.append((slope, y_inter))
        else:
            right.append((slope, y_inter))
    
    left_avg = np.average(left, axis=0)  
    right_avg = np.average(right, axis=0)
    left_line = calculate_coordinates(image, left_avg)
    right_line = calculate_coordinates(image, right_avg)
    
    return np.array([left_line, right_line])

In [18]:
def visualise_lines(image, lines):
    lines_visualised = np.zeros_like(image)
    
    if lines is not None:
        for x1,y1,x2,y2 in lines:
            cv.line(lines_visualised, (x1,y1), (x2,y2), (0,255,0), 5)
    return lines_visualised
            

In [19]:
def calculate_coordinates(frame, parameters):
    try:
        slope, intercept = parameters
    except TypeError:
        slope, intercept = 0.001, 0 
   
    y1 = frame.shape[0]
    y2 = int(y1 - 150)
    x1 = int((y1 - intercept) / slope)
    x2 = int((y2 - intercept) / slope)
    return np.array([x1, y1, x2, y2])

In [20]:
cap = cv.VideoCapture('./videos/test2.mp4')

In [21]:
while cap.isOpened():
    _,frame = cap.read()
    
    canny = do_canny(frame)
    cv.imshow("canny", canny)
    
    segment = do_segment(canny)
    hough = cv.HoughLinesP(segment, 2, np.pi/180, 100, np.array([]), minLineLength = 120, maxLineGap = 40)
    lines = calculate_lines(frame, hough)
    lines_visualised = visualise_lines(frame, lines)
    cv.imshow("hough", lines_visualised)
    output = cv.addWeighted(frame, 0.9, lines_visualised, 1,1)
    
    cv.imshow("output", output)
    
    if cv.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

TypeError: 'NoneType' object is not iterable